In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

csv = 'D:\ISTD 2017-2\Singapore MRT Optimization\singapore-pop.csv'
data = pd.read_csv(csv)

print (data.info())

def distance(p1,p2):
    return ((p2.x - p1.x)**2 + (p2.y - p1.y)**2)**0.5

def distance_b(x1,y1,x2,y2):
    return ((x2-x2)**2 + (y2-y1)**2)**0.5

def generate_random_stations(number):
    lat_bounds = [1.26744,1.356685]
    long_bounds = [103.805866,103.833332]
    
    lat = np.random.randint((lat_bounds[1]-lat_bounds[0])*(10**6),size=number)
    long = np.random.randint((long_bounds[1]-long_bounds[0])*(10**6),size=number)
    output = None
        
    for i in range(number):
        if output is None:
            output = np.append(lat_bounds[0]+(1.0*lat[i]*(10**-6)),long_bounds[0]+(1.0*long[i]*(10**-6)))
        else:
            output = np.append(output,[lat_bounds[0]+(1.0*lat[i]*(10**-6)),long_bounds[0]+(1.0*long[i]*(10**-6))])

    return output

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70203 entries, 0 to 70202
Data columns (total 12 columns):
X             70203 non-null float64
Y             70203 non-null float64
REGION_N      70203 non-null object
HDB           70203 non-null float64
ONE_TO_TWO    70203 non-null float64
THREE_RM      70203 non-null float64
FOUR_RM       70203 non-null float64
FIVE_RM_EX    70203 non-null float64
CONDOS_OTH    70203 non-null float64
LANDED_PRO    70203 non-null float64
OTHERS        70203 non-null float64
total_pop     70203 non-null int64
dtypes: float64(10), int64(1), object(1)
memory usage: 6.4+ MB
None


In [7]:
from scipy.sparse.csgraph import dijkstra
from scipy.optimize import minimize
from scipy.spatial import KDTree

ml = [1.2867888,103.8545000]

def compute_total_distance(stations):
    stations = np.reshape(stations,(len(stations)//2,2))

    tree = KDTree(stations)

    d,i = tree.query(ml)
    
    total_distance = 0
    graph = None
    
    for i in range(len(stations)):
        line = np.zeros(len(stations))
        
        if i-1 >= 0:
            line[i-1] = distance_b(stations[i][0],stations[i][1],stations[i-1][0],stations[i-1][1])
        if i+1 < len(stations):
            line[i+1] = distance_b(stations[i][0],stations[i][1],stations[i+1][0],stations[i+1][1])
            
        graph = line if graph is None else np.append(graph,line)
            
    graph = np.reshape(graph,(len(stations),len(stations)))            
    distance_map = dijkstra(graph,directed=False)
    
    for index,row in data.iterrows():

        n_d,n_i = tree.query([row['Y'],row['X']])
        commute = 9*n_d + distance_map[i][n_i] + 9*d
        total_distance += commute*(row['total_pop']/1000)
        
    print (total_distance)
        
    return total_distance

In [3]:
total_num = 10
rand_stations = generate_random_stations(total_num)

In [ ]:
o = minimize(compute_total_distance,rand_stations,method='BFGS')
print (o)

new_stations = np.reshape(o.x,len(o.x)//2,2)
plt.scatter(new_stations[:,0],new_stations[:,1])
plt.show()

1309767.78352
1309767.79787
1309767.60841
1309767.78327
1309767.78326
1309767.78342
1309767.78271
1309767.78376
1309767.78346
1309767.78241
1309767.78355
1309767.78466
1309767.80276
1309767.78271
1309767.77792
1309767.75436
1309767.80064
1309767.78307
1309767.77465
1309767.7656
1309767.69681
1309767.78352
1679815.02879
1679815.02879
1679815.02879
1679815.02879
1679815.02844
1679815.02881
1679815.0285
1679815.02962
1679815.02616
1679815.03276
1679815.02861
1679815.03333
1679815.01125
1679815.01685
1679814.94238
1679814.88658
1679815.03791
1679815.00689
1679815.00519
1679814.93875
1679815.02879
1679815.0481
1077650.0317
1077650.0317
1077650.0317
1077650.0317
1077650.03123
1077650.03135
1077650.03159
1077650.03147
1077650.03371
1077650.02706
1077650.08209
1077649.86232
1077650.02928
1077650.04424
1077650.03131
1077650.04163
1077650.01687
1077650.01289
1077650.01928
1077650.00514
1077650.06303
1077650.09506
773080.268343
773080.268343
773080.268343
773080.268343
773080.267184
773080.267937